# House Price regression

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define different `CommandComponent` using YAML
- Create `Pipeline` load these components from YAML

**Motivations** - This notebook explains how to load component via SDK then use these components to build pipeline. We use NYC dataset, build pipeline with five steps, prep data, transform data, train model, predict results and evaluate model performance.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = InteractiveBrowserCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "96aede12-2f73-41cb-b983-6d11a904839b",
        "resource_group": "Hahaha",
        "workspace_name": "Hahaha",
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: C:\Users\zhangzheng.REDMOND\source\repos\AML_hahaha\src\pipelines\.azureml\config.json
Found the config file in: C:\Users\zhangzheng.REDMOND\source\repos\AML_hahaha\src\pipelines\.azureml\config.json


MLClient(credential=<azure.identity._credentials.browser.InteractiveBrowserCredential object at 0x00000235760EBD60>,
         subscription_id=96aede12-2f73-41cb-b983-6d11a904839b,
         resource_group_name=Hahaha,
         workspace_name=Hahaha)
AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/Hahaha/providers/Microsoft.MachineLearningServices/workspaces/Hahaha/computes/cpu-cluster', 'source_path': None, 'base_path': 'c:\\Users\\zhangzheng.REDMOND\\source\\repos\\AML_hahaha\\src\\pipelines\\house_price_xgbregression', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002357B129E50>, 'resource_id': None, 'location': 'southeastasia', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 1200.

# 2. Build pipeline

In [45]:
parent_dir = ""

# 1. Load components
prepare_data = load_component(path=parent_dir + "./prep.yml")
# transform_data = load_component(path=parent_dir + "./transform.yml")
train_model = load_component(path=parent_dir + "./train.yml")
# predict_result = load_component(path=parent_dir + "./predict.yml")
# score_data = load_component(path=parent_dir + "./score.yml")

# 2. Construct pipeline
@pipeline()
def house_price_regression(pipeline_job_input):
    prepare_sample_data = prepare_data(raw_data=pipeline_job_input)
    train_with_sample_data = train_model(
        training_data=prepare_sample_data.outputs.prep_data,
        price_data=prepare_sample_data.outputs.price_data
    )
    return {
        "pipeline_job_prepped_train_data": prepare_sample_data.outputs.prep_data,
        "pipeline_job_prepped_price_data": prepare_sample_data.outputs.price_data,
        "pipeline_job_trained_model": train_with_sample_data.outputs.model_output,
        }


pipeline_job = house_price_regression(
    Input(type="uri_folder", path=parent_dir + "./data/")
)
# demo how to change pipeline output settings
pipeline_job.outputs.pipeline_job_prepped_train_data.mode = "rw_mount"

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

## 3. Submit pipeline job

In [46]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="house_price_regression"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
house_price_regression,jolly_plum_m31gs3c812,pipeline,Preparing,Link to Azure Machine Learning studio


In [47]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: jolly_plum_m31gs3c812
Web View: https://ml.azure.com/runs/jolly_plum_m31gs3c812?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourcegroups/Hahaha/workspaces/Hahaha

Streaming logs/azureml/executionlogs.txt

[2022-08-11 08:51:36Z] Completing processing run id d081302f-bcc6-48b8-9e1d-f25e9a40c8e8.
[2022-08-11 08:51:37Z] Submitting 1 runs, first five are: e0583c79:c1156c5e-ee7e-4c2c-b225-0d745007d4c0
[2022-08-11 08:53:34Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: jolly_plum_m31gs3c812
Web View: https://ml.azure.com/runs/jolly_plum_m31gs3c812?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourcegroups/Hahaha/workspaces/Hahaha


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "message_format": "Pipeline has some failed steps. {0}",
        "message_parameters": {},
        "details": []
    },
    "environment": "southeastasia",
    "location": "southeastasia",
    "time": "2022-08-11T08:53:34.655044Z",
    "component_name": ""
} 

# Next Steps
You can see further examples of running a pipeline job [here](../)